In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # 作業 : (Kaggle)鐵達尼生存預測
 https://www.kaggle.com/c/titanic

 # [作業目標]
 - 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

 # [作業重點]
 - 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5])
 - 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7])
 - 試著回答上述執行結果的觀察

 # 作業1
 * 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，
 與其他類別型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()



,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()


5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


 # 作業2
 * 承上題，三者比較效果何者最好?

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()



0.780004837244799


d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [4]:
# 加上 'Cabin' 欄位的計數編碼
df_group  = df_temp.groupby(['Cabin'])['Name'].agg({'Cabin_Cnt':'size'}).reset_index()
#df_group
df_temp = df_temp.merge(df_group,how='left',on=['Cabin'])
df_temp





d:\acetax\2nd-ml100days\.env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Cnt
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014
5,818,1,373,185,2,1014
6,767,1,109,163,3,2
7,914,1,541,185,3,1014
8,605,0,477,185,3,1014
9,847,0,174,185,0,1014


In [5]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()


0.7856230275549181


d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Cnt
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [6]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸






In [7]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸



In [8]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





ValueError: cannot reindex from a duplicate axis

In [9]:
df_temp = df.reset_index()



In [10]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp['Cabin_Hash'] = df_temp['Cabin'].apply(lambda x: hash(x)%3)





In [11]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df_temp['Cabin'].apply(lambda x: hash(x)%3)





In [12]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





ValueError: cannot reindex from a duplicate axis

In [13]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])

df_tmep.head(5)
# df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





NameError: name 'df_tmep' is not defined

In [14]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])

df_temp.head(5)
# df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [15]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df.head(5)
df_temp.head(5)
# df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [16]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df.head(5)

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [17]:
df_temp.head(5)
# df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [18]:
#df_temp.head(5)
df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3)





ValueError: cannot reindex from a duplicate axis

In [19]:
#df_temp.head(5)
df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x)%3, axis=1)





TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [20]:
#df_temp.head(5)
df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x), axis=1)





TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [21]:
#df_temp.head(5)
df_temp['Cabin_Hash'] = df['Cabin'].apply(lambda x: hash(x))





ValueError: cannot reindex from a duplicate axis

In [22]:
#df_temp.head(5)
 df['Cabin'].apply(lambda x: hash(x))





IndentationError: unexpected indent (<ipython-input-22-d0653d50ddb0>, line 2)

In [23]:
#df_temp.head(5)
df['Cabin'].apply(lambda x: hash(x))





0      -515035403211697205
1     -7828263960998568292
2      -515035403211697205
3      8120042937664578979
4      -515035403211697205
5      -515035403211697205
6      4199789275703367926
7      -515035403211697205
8      -515035403211697205
9      -515035403211697205
10    -6314771635108667135
11      495982096613699647
12     -515035403211697205
13     -515035403211697205
14     -515035403211697205
15     -515035403211697205
16     -515035403211697205
17     -515035403211697205
18     -515035403211697205
19     -515035403211697205
20     -515035403211697205
21     3851734909516591625
22     -515035403211697205
23     4001360519075981179
24     -515035403211697205
25     -515035403211697205
26     -515035403211697205
27    -1979825466105969622
28     -515035403211697205
29     -515035403211697205
              ...         
388    -515035403211697205
389    -515035403211697205
390    7883242739618312251
391   -2208718750794113674
392    -515035403211697205
393    -515035403211697205
3

In [24]:
#df_temp.head(5)
df.reset_index()['Cabin'].apply(lambda x: hash(x))





0       -515035403211697205
1      -7828263960998568292
2       -515035403211697205
3       8120042937664578979
4       -515035403211697205
5       -515035403211697205
6       4199789275703367926
7       -515035403211697205
8       -515035403211697205
9       -515035403211697205
10     -6314771635108667135
11       495982096613699647
12      -515035403211697205
13      -515035403211697205
14      -515035403211697205
15      -515035403211697205
16      -515035403211697205
17      -515035403211697205
18      -515035403211697205
19      -515035403211697205
20      -515035403211697205
21      3851734909516591625
22      -515035403211697205
23      4001360519075981179
24      -515035403211697205
25      -515035403211697205
26      -515035403211697205
27     -1979825466105969622
28      -515035403211697205
29      -515035403211697205
               ...         
1279    -515035403211697205
1280    -515035403211697205
1281    7883242739618312251
1282   -2208718750794113674
1283    -51503540321

In [25]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df.reset_index()['Cabin'].map(lambda x : hash(x) % 3)

In [26]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df.reset_index()['Cabin'].map(lambda x : hash(x) % 3)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7744056909263005


d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,1
1,286,0,816,106,0,2
2,523,0,914,185,3,1
3,422,0,65,70,3,1
4,22,1,649,185,3,1


In [27]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_group  = df_temp.groupby(['Cabin'])['Name'].agg({'Cabin_Cnt':'size'}).reset_index()
#df_group
df_temp = df_temp.merge(df_group,how='left',on=['Cabin'])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7755545365660598


d:\acetax\2nd-ml100days\.env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\acetax\2nd-ml100days\.env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Cnt
0,155,1,720,185,3,1,1014
1,286,0,816,106,0,2,2
2,523,0,914,185,3,1,1014
3,422,0,65,70,3,1,2
4,22,1,649,185,3,1,1014
